In [1]:
import os 
import zipfile 
from scipy.io import loadmat 
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import io
from PIL import Image
import tensorflow as tf 
from tensorflow import keras
import sys
from tqdm import tqdm
import random
from sklearn.model_selection import train_test_split
import pywt 


SEED = 123
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)


dataset_current_folder = "../training_set.zip" # where the zip is
dataset_folder = "C://Users//simon//Desktop//AppliedAI-project" # where I want the dataset - avoid the current folder as Git doesn't allow huge uploads

with zipfile.ZipFile(dataset_current_folder, 'r') as zip: # extract the zip file into the desired folder 
    zip.extractall(dataset_folder)


TEST = 0 # 0 if not testing (set not annotated)


def load_data(sample_prefix, input_dir):    # everything is returned as a numpy array which is easier to manipulate
    
        peak_filepath = os.path.join(input_dir, sample_prefix + '_rpk.mat')
        signal_filepath = os.path.join(input_dir, sample_prefix + '.mat')
        if os.path.isfile(peak_filepath):
            mat_file = loadmat(peak_filepath)
            peaks = np.array(mat_file['rpeaks'],dtype=np.int64)
        if os.path.isfile(signal_filepath):
            mat_file = loadmat(signal_filepath)
            signal = np.asarray(mat_file['ecg'] )

        return peaks, signal

ids = list()                # Id of samples 
rpeaks = list()             # detected peaks of the signal 
ecg_signals = list()        # .mat ecg signal 
frequencies = list()        # sample frequency of the ecg signal 


for f in os.listdir(dataset_folder):
  if f.lower().endswith('.mat'):
    id = f[:4]
    if id not in ids:
      ids.append(id)
      sample_prefix = f[:8]
      peak, signal = load_data(sample_prefix, dataset_folder)
      rpeaks.append(peak)
      ecg_signals.append(signal)
      frequencies.append(int(sample_prefix[5:]))
cols = ["sigId","ecg_lead_1","ecg_lead_2","peaks","frequencies"]

# ecg signals is 105 rows [,,,,,]

first_lead_signals = []
second_lead_signals = []

for signal in ecg_signals:
    first_lead_signals.append(signal[:,0].tolist())    # converting the array to list as list of array is deprecated 
    second_lead_signals.append(signal[:,1].tolist())

df = pd.DataFrame(data =[ids,first_lead_signals,second_lead_signals,rpeaks,frequencies]).T
df.columns = cols

# transform peaks 
for id in tqdm(df.index.tolist()):
    peaks = df.iloc[id]['peaks']
    p_list = list()
    for p in peaks:
        p_list.append(p[0])
    df.iloc[id]['peaks'] = np.asarray(p_list).astype(np.int64)

df.head()

100%|██████████| 105/105 [00:00<00:00, 1299.35it/s]


,sigId,ecg_lead_1,ecg_lead_2,peaks,frequencies
0,S001,"[0.0, 0.04, 0.03, 0.0, 0.03, 0.09, 0.18, 0.14,...","[0.08, 0.07, 0.1, 0.06, 0.06, 0.03, 0.1, 0.21,...","[29, 110, 191, 272, 353, 433, 514, 595, 676, 7...",128
1,S002,"[-0.035, -0.045, -0.025, -0.035, -0.045, -0.05...","[-0.095, -0.105, -0.095, -0.095, -0.115, -0.09...","[48, 153, 243, 352, 440, 547, 636, 742, 831, 9...",128
2,S003,"[-0.56, -0.56, -0.55, -0.47, -0.53, -0.47, -0....","[0.43, 0.56, 0.6, 0.41, 0.54, 0.48, 0.56, 0.46...","[91, 209, 326, 394, 537, 653, 745, 872, 984, 1...",128
3,S004,"[-0.46, -0.49, -0.52, -0.58, -0.62, -0.69, -0....","[0.56, 0.61, 0.66, 0.66, 0.63, 0.66, 0.59, 0.5...","[98, 223, 349, 474, 599, 726, 853, 980, 1116, ...",128
4,S005,"[-0.27, -0.17, -0.13, -0.23, -0.18, -0.23, -0....","[-0.02, -0.04, -0.01, -0.01, -0.02, -0.06, 0.0...","[27, 127, 225, 324, 423, 523, 623, 722, 822, 9...",128


In [3]:

from scipy.signal import resample

ids_128 = df[df['frequencies'] == 128].index.tolist()
ids_to_resample = df[df['frequencies'] != 128].index.tolist() 
resampled_len = len(df.iloc[ids_128[0]]['ecg_lead_1'])

for id in tqdm(ids_to_resample):
    row = df.iloc[id]
    sampled_len = len(row['ecg_lead_1'])

    # first lead
    signal = np.asarray(row['ecg_lead_1']).astype(np.float32)
    resampled_1 = resample(signal,resampled_len)
    
    # second lead
    
    signal = np.asarray(row['ecg_lead_2']).astype(np.float32)
    resampled_2 = resample(signal,resampled_len)
    
    df.iloc[id]['ecg_lead_1'] = resampled_1.tolist()
    df.iloc[id]['ecg_lead_2'] = resampled_2.tolist()
    
    for i,p in enumerate(list(df.iloc[id]['peaks'])):
        df.iloc[id]['peaks'][i] = int(resampled_len * p/sampled_len)


100%|██████████| 40/40 [00:04<00:00,  8.69it/s]


In [4]:
#Preprocessing used in practical lessons
from scipy.signal import resample, butter, lfilter, iirnotch

def notch_filter(cutoff, fs, q=30):
    nyq = 0.5*fs
    freq = cutoff/nyq
    b, a = iirnotch(freq, q)
    return b, a


def bandpass_filter(data, filter_order=6, lowcut = .8, highcut = 45, signal_freq=128):
    """
    Method responsible for creating and applying Butterworth filter.
    :param deque data: raw data
    :param float lowcut: filter lowcut frequency value
    :param float highcut: filter highcut frequency value
    :param int signal_freq: signal frequency in samples per second (Hz)
    :param int filter_order: filter order
    :return array: filtered data
    """
    powerline = 60
    nyquist_freq = 0.5 * signal_freq
    low = lowcut / nyquist_freq
    high = highcut / nyquist_freq
    b, a = butter(filter_order, [low, high], btype="band")
    y = lfilter(b, a, data,zi = None)
    #b,a = notch_filter(powerline,signal_freq)
    #y = lfilter(b,a,y)
    return y



filtered_df = df.copy(deep= True)

ids = filtered_df.index.tolist()

for id_ in tqdm(ids):
    
    row = filtered_df.iloc[id_]
    first = row['ecg_lead_1']
    second = row['ecg_lead_2']
    filtered_first = np.asarray(bandpass_filter(first))
    filtered_second =  np.asarray(bandpass_filter(second))

    filtered_df.iloc[id_]['ecg_lead_1'] = pd.Series(filtered_first)
    filtered_df.iloc[id_]['ecg_lead_2'] = pd.Series(filtered_second)

100%|██████████| 105/105 [00:01<00:00, 53.42it/s]


In [5]:
import heartpy as hp

fs = 128
time_window = 10
samples = fs * time_window ##

for id in tqdm(filtered_df.index.tolist()):
    row = filtered_df.loc[id]
    first_lead = row['ecg_lead_1']
    second_lead = row['ecg_lead_2']
    l_ = len(first_lead)
    scaled_1 = list()
    scaled_2 = list()
#    
    for i in range(int(l_/samples)):
        to_scale_1 = first_lead[i*samples:i*samples+samples] 
        to_scale_2 = second_lead[i*samples:i*samples+samples]
#        
        scaled_1 += list(hp.scale_data(to_scale_1,lower=0,upper=1))
        scaled_2 += list(hp.scale_data(to_scale_2,lower=0,upper=1))
 ##   
    filtered_df.loc[id]['ecg_lead_1'] = scaled_1
    filtered_df.loc[id]['ecg_lead_2'] = scaled_2

100%|██████████| 105/105 [00:14<00:00,  7.35it/s]


In [6]:
signal_ids = filtered_df.index.tolist()
RR_dict = {}

for id_ in tqdm(signal_ids):  
  sig_Id = df.iloc[id_]['sigId']
  RR_dict[sig_Id] = {}
  RR_dict[sig_Id]['RR_distances'] = list()
  
  peaks = df.iloc[id_]['peaks']

  for i,p in enumerate(peaks[1:-1]):
    RR_distance = p-peaks[i]
    if(RR_distance < 250): # discard outliers
      RR_dict[sig_Id]['RR_distances'].append(RR_distance)

  RR_dict[sig_Id]['RR_distances'] = np.asarray(RR_dict[sig_Id]['RR_distances'])


  RR_dict[sig_Id]['Avg_RR_distance'] = RR_dict[sig_Id]['RR_distances'].mean()



RR_df = pd.DataFrame.from_dict(RR_dict,orient="index")
RR_df.tail()

100%|██████████| 105/105 [00:00<00:00, 591.86it/s]


,RR_distances,Avg_RR_distance
S114,"[117, 116, 115, 112, 113, 114, 114, 112, 116, ...",114.914745
S115,"[113, 118, 80, 151, 118, 123, 76, 152, 119, 11...",89.041006
S116,"[143, 145, 141, 146, 143, 142, 144, 142, 143, ...",122.611082
S117,"[142, 143, 147, 148, 145, 139, 148, 139, 140, ...",106.363048
S118,"[85, 84, 86, 81, 82, 85, 81, 83, 83, 82, 84, 8...",93.967755


In [7]:
import heartpy as hp 

classes = np.array(["N","S","V"])

patch_length = 350 

def convert_to_one_hot(label):
    return np.array(classes == label,dtype=np.float32)

def create_patch_dataset(df):
    
    dataset_dict = {}
    ids = df['sigId']
   
    for id in ids:
        row = df[df['sigId'] == id]
        sigId = row['sigId'].values[0]
        peaks = row['peaks'].values[0]
        first_lead_signal = row['ecg_lead_1'].values[0]
        avg_RR = RR_df.loc[sigId]['Avg_RR_distance'] # T for the signal 

        for i,peak in enumerate(peaks):
            
            stringIdx = str(i)    
            dataset_dict[stringIdx] = {}
            dataset_dict[stringIdx]['sigId'] = sigId
            dataset_dict[stringIdx]["first_lead"] = list()
            size = list(range(int(peak-(avg_RR)),int(peak+avg_RR)))

            for s in size:
                if(s < 0 or s >= len(first_lead_signal)):
                    dataset_dict[stringIdx]["first_lead"].append(0.5)
                else:
                    dataset_dict[stringIdx]["first_lead"].append(first_lead_signal[s])
            
            first_lead = dataset_dict[stringIdx]["first_lead"][:]
            
            first_lead = resample(first_lead,patch_length)
            
            dataset_dict[stringIdx]["first_lead"] = np.asarray(first_lead[:])
            

    dataset_df = pd.DataFrame.from_dict(dataset_dict,orient="index")
    return dataset_df

In [8]:

def create_input(df):
    x = list()
    
    for id in tqdm(df.index.tolist()):
        row = df.loc[id]
        x.append(np.transpose(np.asarray([row['first_lead']]).astype(np.float32)))

    x = np.asarray(x).astype(np.float32)
    
    return x

In [9]:
import keras
from keras import layers

# This is the size of our encoded representations
encoding_dim = 40 # 350/40 compression factor

input_shape = patch_length

input = keras.Input(shape=input_shape,name='Input_layer')

# encoding 

dense1 = layers.Dense(512,activation='relu',name='Hidden_1')(input)
dropout1 = layers.Dropout(0.1,name='Dropout_1')(dense1)
dense2 = layers.Dense(256,activation='relu',name='Hidden_2')(dropout1)

encoded = layers.Dense(encoding_dim, activation='sigmoid',name='Encoded_input')(dense2)

dense3 = layers.Dense(256,activation='relu',name='Hidden_3')(encoded)
dropout2 = layers.Dropout(0.1,name='Dropout_2')(dense3)
dense4 = layers.Dense(512,activation='relu',name='Hidden_4')(dropout2)

# decoding

decoded = layers.Dense(input_shape, activation='sigmoid',name='Decoded_input')(dense4)

# This model maps an input to its reconstruction
autoencoder = keras.Model(input, decoded)

# This model maps an input to its encoded representation
encoder = keras.Model(input, encoded)

In [10]:
# load the model 

autoencoder.load_weights('./models/autoencoder/autoencoder_weights')

In [11]:
def encode_input(encoder,x_input):
    transformed = encoder.predict(x_input)
    return np.array(transformed)

In [12]:
# load the classifier

from sklearn.svm import SVC
import pickle

path = os.path.join(os.getcwd(),"models","svc")

pkl_filename = "svc.pkl"

with open(os.path.join(path,pkl_filename), 'rb') as file:
    sv = pickle.load(file)


In [114]:
from scipy.io import savemat
classes = np.array(["N","S","V"])

for id in tqdm(filtered_df.index.tolist()[:4]):
    row = filtered_df.loc[id].to_frame().T
    sigId = row['sigId'].values[0]
    patches_df = create_patch_dataset(row)
    predicted_labels = list()
    x_ = patches_df['first_lead']
    x_input = np.array(x_.values.tolist())
    x_encoded = encoder.predict(x_input)
    predictions = sv.predict(x_encoded)

    r = df[df['sigId'] == id]
    freq = row['frequencies'].values[0]
    

    file_path = os.path.join(os.getcwd(),'output')
    file_name = sigId + '_' + str(freq) + '_ann.mat'

    for p in predictions:
        predicted_labels.append(classes[p])
    
    mat_dic = {}
    mat_dic['labels'] = predicted_labels[:]

    savemat(os.path.join(file_path,file_name), mat_dic)



    

100%|██████████| 4/4 [00:08<00:00,  2.20s/it]


2851

(2851, 350)